<a href="https://colab.research.google.com/github/DavidB363/file-store/blob/master/Lab5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# My Recommender System

import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [16]:
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv')
df=df.drop(df.columns[0],1)
print(df)

       -7.82   8.79  -9.66  -8.16  -7.52  ...  99.22  -5.63  99.23  99.24  99.25
0       4.08  -0.29   6.36   4.37  -2.38  ...  -2.14   3.06   0.34  -4.32   1.07
1      99.00  99.00  99.00  99.00   9.03  ...  99.00  99.00  99.00  99.00  99.00
2      99.00   8.35  99.00  99.00   1.80  ...  99.00  99.00  99.00  99.00  99.00
3       8.50   4.61  -4.17  -5.39   1.36  ...   1.55   3.11   6.55   1.80   1.60
4      -6.17  -3.54   0.44  -8.50  -7.09  ...  -3.35   0.05  -9.08  -5.05  -3.45
...      ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24977   0.44   7.43   9.08   2.33   3.20  ...   9.03   6.55   8.69   8.79   7.43
24978   9.13  -8.16   8.59   9.08   0.87  ...  -8.20  -7.23  -8.59   9.13   8.45
24979  99.00  99.00  99.00  99.00  -7.77  ...  99.00  99.00  99.00  99.00  99.00
24980  99.00  99.00  99.00  99.00  -9.71  ...  99.00  99.00  99.00  99.00  99.00
24981   2.43   2.67  -3.98   4.27  -2.28  ...  99.00  99.00  99.00  99.00  99.00

[24982 rows x 100 columns]


In [17]:
arr = df.values
print(arr.shape)
#print(arr_full)

(24982, 100)


In [18]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1810381 (1810381,)


In [0]:

def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [20]:
new_arr, idx = replace(arr, 0.1)
print(idx)

(array([ 1438, 20307, 10013, ...,  6562, 24678,  9793]), array([21, 55,  3, ..., 25, 64, 37]))


In [21]:
arr[idx[0][0], idx[1][0]]

1.75

In [22]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [0]:
# Set the number of latent factors to 2.

n_latent_factors = 2

user_ratings = new_arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [24]:
latent_item_features

array([[0.643, 0.015],
       [0.207, 0.929],
       [0.093, 0.632],
       [0.243, 0.633],
       [0.84 , 0.211],
       [0.705, 0.459],
       [0.776, 0.442],
       [0.974, 0.407],
       [0.764, 0.283],
       [0.364, 0.333],
       [0.393, 0.065],
       [0.285, 0.822],
       [0.384, 0.047],
       [0.012, 0.565],
       [0.856, 0.444],
       [0.866, 0.866],
       [0.261, 0.143],
       [0.619, 0.294],
       [0.126, 0.034],
       [0.437, 0.832],
       [0.336, 0.724],
       [0.205, 0.609],
       [0.33 , 0.998],
       [0.043, 0.932],
       [0.969, 0.087],
       [0.735, 0.684],
       [0.969, 0.116],
       [0.985, 0.849],
       [0.197, 0.039],
       [0.798, 0.294],
       [0.185, 0.066],
       [0.857, 0.775],
       [0.355, 0.435],
       [0.663, 0.015],
       [0.889, 0.081],
       [0.438, 0.72 ],
       [0.295, 0.707],
       [0.019, 0.938],
       [0.668, 0.127],
       [0.792, 0.074],
       [0.549, 0.659],
       [0.58 , 0.67 ],
       [0.436, 0.319],
       [0.3

In [25]:
latent_user_preferences

array([[0.231, 0.757],
       [0.193, 0.766],
       [0.468, 0.609],
       ...,
       [0.788, 0.984],
       [0.085, 0.908],
       [0.428, 0.13 ]])

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
#def train(user_id, item_id, rating, alpha=0.01):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    



#def sgd(iterations=300000):
def sgd(iterations=10):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = [] 
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if (rating!=99):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        # if (iteration % 10000) == 0:
        if (iteration % 1) == 0:
            print(mse)

In [27]:
sgd(10)  # Note how the MSE decreases with the number of iterations

26.220265341629485
25.26066709965554
24.973258613193607
24.755771535175594
24.544867326256636
24.332221123695177
24.117058721389082
23.900102740976944
23.68245422985053
23.465279365922687


In [28]:
sgd(100)

23.249669259784003
23.036544316490808
22.826571144021315
22.620082933519928
22.417000618401644
22.21675473112159
22.018210681210693
21.819605161964564
21.61851006090636
21.41185358483025
21.19604562501521
20.96727078431079
20.722015153660276
20.45785882490912
20.174468307713738
19.87455597008583
19.56439693323259
19.2534545550484
18.952921498828452
18.673512154080356
18.423322571608647
18.206597124397668
18.023739285756537
17.87226933467493
17.748116821554795
17.64673665516823
17.56382916006973
17.495683424326096
17.43926462583435
17.392166436367006
17.352511435251493
17.318844337905166
17.290037295914683
17.265212804455302
17.243683636694687
17.224907106541117
17.20845063921423
17.19396597919988
17.1811698845833
17.16982965063963
17.159752214960154
17.150775915960867
17.142764218753214
17.135600902963546
17.129186340644832
17.123434590698647
17.118271108327665
17.113630920850508
17.109457159861684
17.105699868007
17.10231501936268
17.099263707603395
17.09651146731801
17.09402770207537

KeyboardInterrupt: ignored